## DQN sobre ambientes de Classic Control

https://gymnasium.farama.org/environments/classic_control/

### Imports

In [1]:
import torch
import numpy as np
import random
import numpy as np
import gymnasium

### Seteamos los devices

In [2]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Running on {DEVICE}")
print("Cuda Available:", torch.cuda.is_available())

Running on cpu
Cuda Available: False


### Seteo de seeds
Siempre es buena práctica hacer el seteo de seeds para la reproducibilidad de los experimentos

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

### Creamos el ambiente y probamos algunas de sus funciones.

En este caso elegimos el CartPole pero pueden cambiarlo en la variable *ENV_NAME*.
El ambiente CartPole tiene la ventaja de que las recompensas son positivas y es mas fácil propagar estas hacia los estados iniciales. Mountain Car tiene como recompensa -1 por cada paso que damos y esta limitado a 200 pasos.

In [4]:
ENVS = ["MountainCar-v0", "CartPole-v1", "Acrobot-v1"]
ENV_NAME = ENVS[1]

env = gymnasium.make(ENV_NAME, render_mode="rgb_array")

print("Actions #",env.action_space)
print(env.observation_space.shape)
env.reset()
next_state, reward, terminated, truncated, info = env.step(action=0)

print(f"{next_state.shape},\n {reward},\n {terminated},\n {info}")

Actions # Discrete(2)
(4,)
(4,),
 1.0,
 False,
 {}


### Seteamos los hyperparámetros

In [5]:
def process_state(obs, device):
    return torch.tensor(obs, device=device).unsqueeze(0)

#Hiperparámetros de entrenamiento del agente DQN
TOTAL_STEPS = 1000000
EPISODES = 1000
STEPS = 200

EPSILON_INI = .9
EPSILON_MIN = 0.05
EPSILON_DECAY = 50000
EPISODE_BLOCK = 100
EPSILON_TIME = 100000

BATCH_SIZE = 128
BUFFER_SIZE = 10000

GAMMA = 0.99
LEARNING_RATE = 1e-4

### Creamos el ambiente que vamos a estar usando para el entrenamiento

In [6]:
env = gymnasium.make(ENV_NAME)
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

print(f"Input dim: {input_dim}, Output dim: {output_dim}")

Input dim: 4, Output dim: 2


In [7]:
from actor_critic_models import ActorModel, CriticModel
from actor_critic_agent import ActorCriticAgent

actor = ActorModel(input_dim, output_dim).to(DEVICE)
critic = CriticModel(input_dim).to(DEVICE)

agent = ActorCriticAgent(env, actor, process_state, BUFFER_SIZE, BATCH_SIZE, 
                LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI, 
                epsilon_f=EPSILON_MIN, epsilon_decay = EPSILON_DECAY, 
                episode_block = EPISODE_BLOCK, device=DEVICE, critic_model=critic)

c:\Users\juan1\anaconda3\envs\TallerIA\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [8]:
# from dqn_model import DQN_Model
# from dqn_agent import DQNAgent
# net = DQN_Model(input_dim, output_dim).to(DEVICE)

# agent = DQNAgent(env, net, process_state, BUFFER_SIZE, BATCH_SIZE, 
#                 LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI, 
#                 epsilon_f=EPSILON_MIN, epsilon_anneal_time=EPSILON_TIME,
#                 epsilon_decay = EPSILON_DECAY, episode_block = EPISODE_BLOCK, device=DEVICE, second_model_update=300)

In [9]:
# from dqn_model import DQN_Model
# from double_dqn_agent import DoubleDQNAgent
# net = DQN_Model(input_dim, output_dim).to(DEVICE)
# net2 = DQN_Model(input_dim, output_dim).to(DEVICE)

# agent = DoubleDQNAgent(env, net, net2, process_state, BUFFER_SIZE, BATCH_SIZE, 
#                 LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI, epsilon_f=EPSILON_MIN, 
#                 epsilon_decay = EPSILON_DECAY, episode_block = EPISODE_BLOCK, device=DEVICE)

In [10]:
rewards = agent.train(EPISODES, TOTAL_STEPS, use_wandb=False)

  0%|          | 1/1000 [00:00<14:54,  1.12episode/s]c:\Users\juan1\Ort\Agentes Inteligentes\reinforcement-learning\Atari\actor_critic_agent.py:61: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(rewards + self.gamma * next_state_val.detach(), state_val)
  1%|          | 9/1000 [00:01<01:24, 11.70episode/s]

Episode 0 - Avg. Reward over the last 100 episodes 15.0 epsilon 0.9 total steps 15


 10%|█         | 104/1000 [00:06<00:44, 20.00episode/s]

Episode 100 - Avg. Reward over the last 100 episodes 13.48 epsilon 0.9 total steps 1363


 20%|██        | 203/1000 [00:10<00:34, 23.17episode/s]

...

 30%|███       | 305/1000 [00:15<00:31, 22.38episode/s]

Episode 300 - Avg. Reward over the last 100 episodes 9.38 epsilon 0.9 total steps 3258


 34%|███▎      | 337/1000 [00:16<00:33, 19.91episode/s]

tensor([[nan, nan]], grad_fn=<SoftmaxBackward0>)


AttributeError: 'NoneType' object has no attribute 'item'

### Graficamos las recompensas obtenidas durante el entrenamiento 

In [ ]:
import matplotlib.pyplot as plt

average_range = EPISODE_BLOCK
episode_ticks = int(len(rewards) / average_range)

avg_rewards = np.array(rewards).reshape((episode_ticks, average_range))
avg_rewards = np.mean(avg_rewards, axis=1)

plt.plot(range(len(avg_rewards)), avg_rewards)

### Creamos un video para ver la performance del agente

In [ ]:
import glob
from gymnasium.wrappers.record_video import RecordVideo
from IPython.display import HTML
from IPython import display as ipythondisplay
import io
import base64

def show_video():
  """
  Utility function to enable video recording of gym environment and displaying it
  To enable video, just do "env = wrap_env(env)""
  """
  mp4list = glob.glob('./video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")


def wrap_env(env):
  """
  Wrapper del ambiente donde definimos un Monitor que guarda la visualizacion como un archivo de video.
  """

  #env = Monitor(env, './video', force=True)
  env = RecordVideo(env,video_folder='./videos', name_prefix='video', episode_trigger=lambda x: x % 2 == 1)
  return env

In [ ]:
env = wrap_env(gymnasium.make(ENV_NAME, render_mode="rgb_array"))
observation,_ = env.reset()

while True:
    env.render()

    action = agent.select_action(process_state(observation, DEVICE), train=False)
    observation, reward, done, truncated, _ = env.step(action.item())

    if done:
      break

# Cerramos la conexion con el Monitor de ambiente y mostramos el video.
env.close()
show_video()

del env

In [ ]:
agent.env = gymnasium.make(ENV_NAME, render_mode="rgb_array")
agent.env.reset()
agent.make_video("video_test.mp4", True)